# **Projek Akhir Praktikum Data Science**

> Analisis Sentimen Mengenai Vaksin COVID-19 Di Indonesia Menggunakan Metode Naive Bayes Classifier dan NLP Pada Sosial Media Twitter

**Oleh Kelompok 2 :**
1. Hazlan Muhammad Qodri (123190080)
2. Elisia Dwi Rahayu (123190062)
3. Shania Septika Inayasari (123190055)

**Penjelasan Projek :**

Adapun pada penelitian menekankan kepada sentimen masyarakat terhadap mengenai vaksin COVID-19. Proses analisisnya akan dilakukan berdasarkan tweet yang menyertakan tagar vaksin dan pencarian di twitter dengan keyword vaksin covid 19.

# **1. Scraping Data from Twitter**

In [ ]:
%pip install tweepy

In [32]:
import tweepy
import csv
import os
from datetime import timedelta, datetime
import pandas as pd

#### Secret Env Variable

In [33]:
consumer_key = os.environ.get('consumer_key')
consumer_secret = os.environ.get('consumer_key_secret')
access_token = os.environ.get('access_token')
access_token_secret = os.environ.get('access_token_secret')
bearer_token = os.environ.get('bearer_token')

In [34]:
# token for twitter developers
client = tweepy.Client(
    bearer_token=bearer_token,
    consumer_key=consumer_key,
    consumer_secret=consumer_secret,
    access_token=access_token,
    access_token_secret=access_token_secret,
)

def scrap_tweets(search_words, date_since):
    # Membuat Kolom Untuk Di Export Ke Excel
    db_tweets = pd.DataFrame(columns=[
        'username', 'acctdesc', 'location', 'following',
        'totaltweets', 'usercreatedts', 'tweetcreatedts',
        'retweetcount', 'text', 'hashtags', 'followers',
    ])
    print(db_tweets)

    # Melakukan Query Pencarian Data Tweet Sesuai Kata Kunci Dan Tanggal
    # tweets = client.search_recent_tweets(query=search_words,expansions=None, max_results=100, since_id=date_since)
    tweet_list = []
    for tweet in client.sea(q="iphone", lang="en"):
        print(tweet.text)
    print(tweet_list)

    # Merubah Kumpulan Item Hasil Tweet Menjadi Kumpulan Data Dalam Array List
    

    # # Melakukan Perulangan Untuk Data Tweet Untuk Dimasukkan Kedalam CSV
    for tweet in tweet_list:
        username = tweet.user.screen_name
        acctdesc = tweet.user.description
        location = tweet.user.location
        following = tweet.user.friends_count
        followers = tweet.user.followers_count
        totaltweets = tweet.user.statuses_count
        usercreatedts = tweet.user.created_at
        tweetcreatedts = tweet.created_at
        retweetcount = tweet.retweet_count
        hashtags = tweet.entities['hashtags']

        try:
            text = tweet.retweeted_status.full_text
        except AttributeError:
            text = tweet.full_text

        # Membuat Array Kumpulan Data Sesuai Kolom Dan Memasukkan Kedalam Array Data Tweet
        ith_tweet = [
            username, acctdesc, location, following, followers, totaltweets,
            usercreatedts, tweetcreatedts, retweetcount, text, hashtags
        ]

        db_tweets.loc[len(db_tweets)] = ith_tweet

    # Export Data Kumpulan Tweet Ke File CSV
    filename = 'covid_vaccine_tweets.csv'
    db_tweets.to_csv(filename, index=False)
    print('Scraping has completed!')


# Date Config
today = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
last_week = datetime.utcnow() - timedelta(7)
last_week = last_week.strftime("%Y-%m-%d %H:%M:%S")

# Searching Word
search_words = "covid indonesia OR kasus covid OR #kasuscovid OR #vaksin OR #waspadacovid"
date_since = last_week

scrap_tweets(search_words, date_since)



Status Failed On, 403 Forbidden
453 - You currently have Essential access which includes access to Twitter API v2 endpoints only. If you need access to this endpoint, you’ll need to apply for Elevated access via the Developer Portal. You can learn more here: https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-leve


In [ ]:
# Using SNSCRAPE
import snscrape.modules.twitter as sntwitter

